In [1]:
# 환자정보, 검색트랜드 파일 불러옴
patient_info = pd.read_csv("coronavirusdataset\\PatientInfo.csv")
search_trend = pd.read_csv("coronavirusdataset\\SearchTrend.csv")

In [2]:
patient_info

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75.0,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31.0,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2.002000e+09,17.0,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9.0,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1.000000e+09,2.0,NaN,2020-01-31,2020-02-24,NaN,released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,7000000009,9651.0,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,14.0,NaN,2020-03-29,NaN,NaN,isolated
3384,7000000010,NaN,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,18.0,NaN,2020-04-03,NaN,NaN,isolated
3385,7000000011,NaN,male,NaN,30s,Korea,Jeju-do,Jeju-do,NaN,contact with patient,NaN,7.000000e+09,5.0,NaN,2020-04-03,NaN,NaN,isolated
3386,7000000012,NaN,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,9.0,NaN,2020-04-03,NaN,NaN,isolated


In [3]:
# patient_info에서 일일 확진자 수와 누적 확진자 수를 구하여 새로운 데이터 프레임 생성
confirmed_data = patient_info["confirmed_date"]
# 확진 날짜를 통해 일일 확진자 컬럼 생성
confirmed_data = patient_info.groupby("confirmed_date").size().reset_index(name="일일확진자수")
confirmed_data

,confirmed_date,일일확진자수
0,2020-01-20,1
1,2020-01-23,1
2,2020-01-26,1
3,2020-01-27,1
4,2020-01-30,3
...,...,...
82,2020-04-25,8
83,2020-04-26,4
84,2020-04-27,7
85,2020-04-28,3


In [4]:
# confirmed_data를 2020년 1월 1일부터 2020년 4월 30일기간 동안 모든 데이터를 생성
date_data = pd.DataFrame()
date_data["confirmed_date"] = pd.date_range("2020-01-01", "2020-04-30")
# date_data와 confirmed_data의 confirmed_date의 자료형을 동일하게 만듬
confirmed_data["confirmed_date"] =  pd.to_datetime(confirmed_data["confirmed_date"])
# confrimed_data와 date_data를 merge
confirmed_data = date_data.merge(confirmed_data, how = 'outer')
# NaN = 확진자가 없는 날짜 : 일일 확진자 수(dayofconfirmed )= 0
confirmed_data.fillna(value = 0, inplace=True)
confirmed_data = confirmed_data.astype({"일일확진자수":'int'})
confirmed_data

,confirmed_date,일일확진자수
0,2020-01-01,0
1,2020-01-02,0
2,2020-01-03,0
3,2020-01-04,0
4,2020-01-05,0
...,...,...
116,2020-04-26,4
117,2020-04-27,7
118,2020-04-28,3
119,2020-04-29,1


In [14]:
# 일일 확진자 수를 통해 누적 확진자 컬럼 생성
confirmed_data["누적확진자수"] = confirmed_data["일일확진자수"].cumsum(axis = 0)
confirmed_data.rename(columns={"confirmed_date":"날짜"}, inplace = True)
confrimed_data = confirmed_data[confirmed_data["날짜"] < "2020-04-30"]
confrimed_data

,날짜,일일확진자수,누적확진자수,search_trend
0,2020-01-01,0,0,0.01699
1,2020-01-02,0,0,0.01827
2,2020-01-03,0,0,0.02845
3,2020-01-04,0,0,0.02136
4,2020-01-05,0,0,0.01927
...,...,...,...,...
115,2020-04-25,8,3370,4.13405
116,2020-04-26,4,3374,3.95342
117,2020-04-27,7,3381,4.11033
118,2020-04-28,3,3384,3.73088


In [15]:
search_trend = search_trend[search_trend["date"] > "2020-00-00"]
search_trend.reset_index(drop=True, inplace=True)
search_trend

,date,cold,flu,pneumonia,coronavirus
0,2020-01-01,0.14454,0.48434,0.44880,0.01699
1,2020-01-02,0.19508,0.85751,0.36471,0.01827
2,2020-01-03,0.19581,0.80443,0.46598,0.02845
3,2020-01-04,0.60343,0.86961,0.49571,0.02136
4,2020-01-05,0.20081,0.84152,0.41644,0.01927
...,...,...,...,...,...
115,2020-04-25,0.12781,0.02709,0.08672,4.13405
116,2020-04-26,0.13363,0.03263,0.08136,3.95342
117,2020-04-27,0.11226,0.04427,0.10390,4.11033
118,2020-04-28,0.11481,0.03981,0.10336,3.73088


In [16]:
confirmed_data["search_trend"] = search_trend["coronavirus"]
confirmed_data

,날짜,일일확진자수,누적확진자수,search_trend
0,2020-01-01,0,0,0.01699
1,2020-01-02,0,0,0.01827
2,2020-01-03,0,0,0.02845
3,2020-01-04,0,0,0.02136
4,2020-01-05,0,0,0.01927
...,...,...,...,...
116,2020-04-26,4,3374,3.95342
117,2020-04-27,7,3381,4.11033
118,2020-04-28,3,3384,3.73088
119,2020-04-29,1,3385,3.78952
